In [1]:
# Notebook for performing technical analysis and backtesting on Sharadar data (alpha)
# Note: code produced with some assistance from Google Gemini

import ta_parser
from simple_prompt_generation import *

In [2]:
import pandas as pd
import numpy as np
import datetime
import abc
import re
import typing
import inspect
import textwrap
import functools
import os
import kaggle_secrets
import google.genai
import IPython.display as ipyd
import inspect
import textwrap
import typing
import google.generativeai
import itertools
import math
import random
import time
import requests
import scipy.optimize


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
def get_secret(key_name="MY_API_KEY"):
    """
    Retrieves a secret from Kaggle Secrets, Google Colab Secrets,
    environment variables, or a .env file, depending on the environment.

    Args:
        key_name (str): The name of the secret.

    Returns:
        str: The secret if found.

    Raises:
        Exception: If the secret is not found in any supported location.
    """
    secret = None

    # 1. Try Kaggle Secrets
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret = user_secrets.get_secret(key_name)
        if secret:
            return secret
    except ImportError:
        pass  # Not on Kaggle, try other methods
    except ValueError:
        pass

    # 2. Try Google Colab Secrets
    try:
        from google.colab import userdata
        secret = userdata.get(key_name)
        if secret:
            return secret
    except ImportError:
        pass  # Not on Colab, try other methods
    except ValueError:
        pass

    # 3. Try environment variables
    secret = os.getenv(key_name)
    if secret:
        return secret

    # 4. Try .env file (for local development)
    try:
        from dotenv import load_dotenv
        load_dotenv()  # Load .env file
        secret = os.getenv(key_name)
        if secret:
            return secret
    except ImportError:
        pass  # dotenv not installed, try other methods

    raise Exception(f"Secret '{key_name}' not found in any supported location.")


In [4]:
gemini_api_key = get_secret("gemini_api_key")
os.environ["GOOGLE_API_KEY"] = gemini_api_key


In [5]:
sharadar_etfs = pd.read_csv("/kaggle/input/sharadar/SFP.csv").sort_values(by=["ticker", "date"]).reset_index(drop=True)

In [6]:
display(sharadar_etfs.head())
display(sharadar_etfs.tail())

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated
0,AAA,2020-09-09,25.10,25.119,25.070,25.070,17327.0,20.970,25.070,2025-03-27
1,AAA,2020-09-10,25.06,25.070,25.046,25.068,23485.0,20.968,25.068,2025-03-27
2,AAA,2020-09-11,25.04,25.050,25.020,25.035,33362.0,20.940,25.035,2025-03-27
3,AAA,2020-09-14,25.01,25.060,25.010,25.020,13146.0,20.928,25.020,2025-03-27
4,AAA,2020-09-15,25.02,25.030,25.010,25.010,12069.0,20.920,25.010,2025-03-27


,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated
13706319,^VIX,2025-04-22,32.61,32.68,30.08,30.57,0.0,30.57,30.57,2025-04-22
13706320,^VIX,2025-04-23,28.75,30.29,27.11,28.45,0.0,28.45,28.45,2025-04-23
13706321,^VIX,2025-04-24,28.69,29.66,26.36,26.47,0.0,26.47,26.47,2025-04-24
13706322,^VIX,2025-04-25,26.22,27.20,24.84,24.84,0.0,24.84,24.84,2025-04-25
13706323,^VIX,2025-04-28,25.75,26.93,24.70,25.15,0.0,25.15,25.15,2025-04-28


In [7]:

factory = ta_parser.FunctionFactory()

In [8]:
df = sharadar_etfs

In [9]:
macd_indicator = factory.parse("MACD(12,26,9)")
print(macd_indicator)
macd_result = macd_indicator.calculate(df)
df = df.join(macd_result)
display(df)

MACD(fast_length=12, slow_length=26, signal_length=9)


,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated,"MACD(12,26,9)[""macd""]","MACD(12,26,9)[""signal""]","MACD(12,26,9)[""histogram""]"
0,AAA,2020-09-09,25.10,25.119,25.070,25.070,17327.0,20.970,25.070,2025-03-27,0.000000,0.000000,0.000000
1,AAA,2020-09-10,25.06,25.070,25.046,25.068,23485.0,20.968,25.068,2025-03-27,-0.000160,-0.000032,-0.000128
2,AAA,2020-09-11,25.04,25.050,25.020,25.035,33362.0,20.940,25.035,2025-03-27,-0.002915,-0.000609,-0.002307
3,AAA,2020-09-14,25.01,25.060,25.010,25.020,13146.0,20.928,25.020,2025-03-27,-0.006238,-0.001734,-0.004503
4,AAA,2020-09-15,25.02,25.030,25.010,25.010,12069.0,20.920,25.010,2025-03-27,-0.009567,-0.003301,-0.006266
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13706319,^VIX,2025-04-22,32.61,32.680,30.080,30.570,0.0,30.570,30.570,2025-04-22,2.672385,3.523936,-0.851551
13706320,^VIX,2025-04-23,28.75,30.290,27.110,28.450,0.0,28.450,28.450,2025-04-23,2.149621,3.249073,-1.099452
13706321,^VIX,2025-04-24,28.69,29.660,26.360,26.470,0.0,26.470,26.470,2025-04-24,1.557602,2.910779,-1.353177
13706322,^VIX,2025-04-25,26.22,27.200,24.840,24.840,0.0,24.840,24.840,2025-04-25,0.945991,2.517821,-1.571831


In [10]:
if False:
    atr_indicator = factory.parse("ATR(4)")
    print(atr_indicator)
    atr_result = atr_indicator.calculate(df)
    df = df.join(atr_result)
    
    
    adx_indicator = factory.parse("ADX(14)")
    print(adx_indicator)
    adx_result = adx_indicator.calculate(df)
    df = df.join(adx_result)
    
    cci_indicator = factory.parse("CCI(25)")
    print(cci_indicator)
    cci_result = cci_indicator.calculate(df)
    df = df.join(cci_result)
    
    cmf_indicator = factory.parse("CMF(9)")
    print(cmf_indicator)
    cmf_result = cmf_indicator.calculate(df)
    df = df.join(cmf_result)
    
    aro_indicator = factory.parse("Aroon(20)")
    print(aro_indicator)
    aro_result = aro_indicator.calculate(df)
    df = df.join(aro_result)
    
    mfi_indicator = factory.parse("MFI(25)")
    print(mfi_indicator)
    mfi_result = mfi_indicator.calculate(df)
    df = df.join(mfi_result)
    
    pct_indicator = factory.parse("PCT(21)")
    print(pct_indicator)
    pct_result = pct_indicator.calculate(df)
    df = df.join(pct_result)
    
    prp_indicator = factory.parse("PRP(25)")
    print(prp_indicator)
    prp_result = prp_indicator.calculate(df)
    df = df.join(prp_result)
    
    lret_indicator = factory.parse("LRET(5)")
    print(lret_indicator)
    lret_result = lret_indicator.calculate(df)
    df = df.join(lret_result)

In [11]:
rsi_screen = factory.parse("RSI(14)>=80")
print(rsi_screen)
rsi_screen_result = rsi_screen.calculate(df)
df = df.join(rsi_screen_result)

Ge(a0=RSI(length=14), a1=80)


In [12]:

df

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated,"MACD(12,26,9)[""macd""]","MACD(12,26,9)[""signal""]","MACD(12,26,9)[""histogram""]",(RSI(14)>=80)
0,AAA,2020-09-09,25.10,25.119,25.070,25.070,17327.0,20.970,25.070,2025-03-27,0.000000,0.000000,0.000000,False
1,AAA,2020-09-10,25.06,25.070,25.046,25.068,23485.0,20.968,25.068,2025-03-27,-0.000160,-0.000032,-0.000128,False
2,AAA,2020-09-11,25.04,25.050,25.020,25.035,33362.0,20.940,25.035,2025-03-27,-0.002915,-0.000609,-0.002307,False
3,AAA,2020-09-14,25.01,25.060,25.010,25.020,13146.0,20.928,25.020,2025-03-27,-0.006238,-0.001734,-0.004503,False
4,AAA,2020-09-15,25.02,25.030,25.010,25.010,12069.0,20.920,25.010,2025-03-27,-0.009567,-0.003301,-0.006266,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13706319,^VIX,2025-04-22,32.61,32.680,30.080,30.570,0.0,30.570,30.570,2025-04-22,2.672385,3.523936,-0.851551,False
13706320,^VIX,2025-04-23,28.75,30.290,27.110,28.450,0.0,28.450,28.450,2025-04-23,2.149621,3.249073,-1.099452,False
13706321,^VIX,2025-04-24,28.69,29.660,26.360,26.470,0.0,26.470,26.470,2025-04-24,1.557602,2.910779,-1.353177,False
13706322,^VIX,2025-04-25,26.22,27.200,24.840,24.840,0.0,24.840,24.840,2025-04-25,0.945991,2.517821,-1.571831,False


In [13]:


# Create and use screeners
top_n_screener = factory.parse("IsTopN(top_n=2, field=LRET(1))")
top_n_result = top_n_screener.calculate(df)
print(top_n_result)
#print("Top N Result:\n", df.sort_index().loc[top_n_result.sort_index()])

#print(top_n_screener)
#percentile_screener = factory.parse("Percentile(percentile=0.5, field=LRET(1))")
#percentile_result = percentile_screener.calculate(df)
#print("Percentile Result:\n", df.loc[percentile_result])

0           False
1           False
2           False
3           False
4           False
            ...  
13706319    False
13706320    False
13706321    False
13706322    False
13706323    False
Name: IsTopN(2,LRET(1)), Length: 13706324, dtype: bool


In [14]:
df

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated,"MACD(12,26,9)[""macd""]","MACD(12,26,9)[""signal""]","MACD(12,26,9)[""histogram""]",(RSI(14)>=80)
0,AAA,2020-09-09,25.10,25.119,25.070,25.070,17327.0,20.970,25.070,2025-03-27,0.000000,0.000000,0.000000,False
1,AAA,2020-09-10,25.06,25.070,25.046,25.068,23485.0,20.968,25.068,2025-03-27,-0.000160,-0.000032,-0.000128,False
2,AAA,2020-09-11,25.04,25.050,25.020,25.035,33362.0,20.940,25.035,2025-03-27,-0.002915,-0.000609,-0.002307,False
3,AAA,2020-09-14,25.01,25.060,25.010,25.020,13146.0,20.928,25.020,2025-03-27,-0.006238,-0.001734,-0.004503,False
4,AAA,2020-09-15,25.02,25.030,25.010,25.010,12069.0,20.920,25.010,2025-03-27,-0.009567,-0.003301,-0.006266,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13706319,^VIX,2025-04-22,32.61,32.680,30.080,30.570,0.0,30.570,30.570,2025-04-22,2.672385,3.523936,-0.851551,False
13706320,^VIX,2025-04-23,28.75,30.290,27.110,28.450,0.0,28.450,28.450,2025-04-23,2.149621,3.249073,-1.099452,False
13706321,^VIX,2025-04-24,28.69,29.660,26.360,26.470,0.0,26.470,26.470,2025-04-24,1.557602,2.910779,-1.353177,False
13706322,^VIX,2025-04-25,26.22,27.200,24.840,24.840,0.0,24.840,24.840,2025-04-25,0.945991,2.517821,-1.571831,False


In [15]:
#factory.parse("Normalize(RSI(18)>80)").calculate(df)

In [16]:
if False:
    df = sharadar_etfs.copy()

    #normalizer = factory.parse("Normalize(RSI(14) > 80)")
    #normed_results = normalizer.calculate(df)
    #print(normed_results)
    
    cutter = factory.parse("AbsMinCutoff(weights=Normalize(RSI(14) > 80), minimum=0.1)")
    cutter_results = cutter.calculate(df)
    print(cutter_results)       

In [17]:

def grid_maximize(func, param_ranges, max_combinations=None, sampling_method="deterministic"):
    """
    Maximizes a function over a grid of parameters with optional sampling,
    with an optional limit on the maximum number of combinations checked.

    Args:
        func: The function to maximize. It should accept named parameters.
        param_ranges: A dictionary where keys are parameter names and values are dictionaries
                      containing 'min', 'max', and 'increment' keys.
        max_combinations: An optional integer specifying the maximum number of combinations to check.
                          If None, all combinations are checked.
        sampling_method: "deterministic" or "random". Determines the sampling method.

    Returns:
        A dictionary containing the maximized parameters and the corresponding function value.
    """

    param_names = list(param_ranges.keys())
    param_values_lists = []

    for param_name in param_names:
        min_val = param_ranges[param_name]['min']
        max_val = param_ranges[param_name]['max']
        increment = param_ranges[param_name]['increment']

        param_values = []
        current_val = min_val
        while current_val <= max_val:
            param_values.append(current_val)
            current_val += increment

        param_values_lists.append(param_values)

    best_params = None
    best_value = float('-inf')

    all_combinations = list(itertools.product(*param_values_lists))
    total_combinations = len(all_combinations)

    if max_combinations is not None and total_combinations > max_combinations:
        if sampling_method == "random":
            sampled_combinations = random.sample(all_combinations, max_combinations)
        elif sampling_method == "deterministic":
            step = math.ceil(total_combinations / max_combinations)
            sampled_combinations = all_combinations[::step]
        else:
            raise ValueError("Invalid sampling_method. Must be 'random' or 'deterministic'.")
    else:
        sampled_combinations = all_combinations

    for param_values in sampled_combinations:
        params = dict(zip(param_names, param_values))

        try:
            value = func(**params)
        except Exception as e:
            print(f"Error evaluating function with params {params}: {e}")
            continue

        if value > best_value:
            best_value = value
            best_params = params

    if best_params is None:
        return None

    return {"params": best_params, "value": best_value}

# Example usage:
def my_function(x, y, z):
    return - (x - 2)**2 - (y + 1)**2 - (z - 3)**2

param_ranges = {
    'x': {'min': 0, 'max': 4, 'increment': 1},
    'y': {'min': -3, 'max': 1, 'increment': 0.5},
    'z': {'min': 1, 'max': 5, 'increment': 2},
}

# Deterministic sampling
result_det = grid_maximize(my_function, param_ranges, max_combinations=10, sampling_method="deterministic")
print("Maximized parameters (deterministic):", result_det)

# Random sampling
result_rand = grid_maximize(my_function, param_ranges, max_combinations=10, sampling_method="random")
print("Maximized parameters (random):", result_rand)

# All combinations
result_all = grid_maximize(my_function, param_ranges)
print("Maximized parameters (all):", result_all)

# Invalid sampling method
try:
    grid_maximize(my_function, param_ranges, max_combinations=10, sampling_method="invalid")
except ValueError as e:
    print(f"Error: {e}")

Maximized parameters (deterministic): {'params': {'x': 2, 'y': -0.5, 'z': 3}, 'value': -0.25}
Maximized parameters (random): {'params': {'x': 1, 'y': -2.0, 'z': 3}, 'value': -2.0}
Maximized parameters (all): {'params': {'x': 2, 'y': -1.0, 'z': 3}, 'value': 0.0}
Error: Invalid sampling_method. Must be 'random' or 'deterministic'.


In [18]:
response = None
system_instruction = """
You speak as a highly skilled code generation assistant specializing in Python and financial data analysis.
You always prioritize using existing code provided in the context.
You must strictly adhere to all revision instructions in subsequent turns. 
Your goal is to generate concise, correct, and efficient Python code snippets.
"""

google.generativeai.configure(api_key=gemini_api_key)
gemini_model = google.generativeai.GenerativeModel(system_instruction=system_instruction)
gemini = gemini_model.start_chat()

In [19]:
module_stub_code = generate_module_code_stubs(ta_parser)

question = """
I am working with a Jupyter notebook that has defined the following classes and functions\n
(note: I am showing stubs here, but the code is largely implemented):

--[
""" + module_stub_code + """
]--

I need a method for the candlestick functions that is of the same form and structure as this one:

    def register_basic_indicators(factory): 
        #Registers basic technical indicator functions with this factory instance.

        # SMA
        sma_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_sma = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_sma) # Referencing global do_calculate_sma
        factory.register(FunctionDefinition("SMA", {"length": sma_length_param}, calculate_sma)) # Use factory.register


        # RSI
        rsi_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_rsi = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_rsi) # Referencing global do_calculate_rsi
        factory.register(FunctionDefinition("RSI", {"length": rsi_length_param}, calculate_rsi)) # Use factory.register


        # ... numerous indicators omitted for brevity

        # Shift
        shift_n_param = ParameterType("integer", min_val=1, max_val=200, default=1)
        shift_series_param = ParameterType("any", default="close")
        calculate_shift = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_shift) # Referencing global do_calculate_shift
        factory.register(FunctionDefinition("Shift", {"series": shift_series_param, "n": shift_n_param}, calculate_shift)) # Use factory.register    

please be sure to register *all* of the do_detect_xxx functions that detect candlesticks
please register them following the same pattern as you see above
alternatively, you can set up a big array / dict / dict of dicts / etc that specify how to register the functions
and iterate through it
please do use CamelCase/capitalized identifiers chosen for brevity for the candlestick matching names
please also keep in mind that it may be necessary to specify multiple parameters to functions
in the future even if it is not required now
"""        
#response = gemini_model.generate_content(question)
#display(ipyd.Markdown(response.text))


In [20]:
def get_url_content(url):
    """
    Fetches the content of a URL and returns it as a string,
    using a standard web browser user-agent.

    Args:
        url (str): The URL to fetch.

    Returns:
        str: The HTML content of the URL as a string, or None if there was an error.
    """
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'  # Example Chrome User-Agent
    headers = {'User-Agent': user_agent}

    try:
        response = requests.get(url, headers=headers, timeout=10)  # Added timeout for robustness
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.text  # .text automatically decodes response.content using best guess encoding (usually UTF-8 for HTML)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {url} - {e}")
        return None

In [29]:
patterns = {
    '!1-2-3 trend change': 'https://thepatternsite.com/123tc.html',
    '2B': 'https://thepatternsite.com/2B.html',
    '2-Close bullish reversal': 'https://thepatternsite.com/2Closebull.html',
    '2-Close bearish reversal': 'https://thepatternsite.com/2Closebear.html',
    '2-dance': 'https://thepatternsite.com/2Dance.html',
    '2-did': 'https://thepatternsite.com/2Did.html',
    '2-tall': 'https://thepatternsite.com/TallDance.html',
    '3-Bar': 'https://thepatternsite.com/3Bar.html',
    '3DC': 'https://thepatternsite.com/3DC.html',
    '3 falling peaks': 'https://thepatternsite.com/3fp.html',
    '3L-R': 'https://thepatternsite.com/3L-R.html',
    '3L-R, Inverted': 'https://thepatternsite.com/3L-Ri.html',
    '3 peaks and domed house': 'https://thepatternsite.com/3peaksdome.html',
    '3-peaks and spike': 'https://thepatternsite.com/MultiPeak2B.html',
    '3-point Channels, drawing': 'https://thepatternsite.com/3PointChannels.html',
    '3 rising valleys': 'https://thepatternsite.com/3rv.html',
    '10 Baggers': 'https://thepatternsite.com/10Baggers.html',
    '12 month moving average': 'https://thepatternsite.com/12MonthMA.html',
    '38% Exit': 'https://thepatternsite.com/38PercentExit.html',
    'ABC correction, simple': 'https://thepatternsite.com/abc.html',
    'AB=CD, bearish': 'https://thepatternsite.com/ABCDBear.html',
    'AB=CD, bullish': 'https://thepatternsite.com/ABCDBull.html',
    'Adam & Adam double bottoms': 'https://thepatternsite.com/aadb.html',
    'Adam & Adam double tops': 'https://thepatternsite.com/aadt.html',
    'Adam & Eve double bottoms': 'https://thepatternsite.com/aedb.html',
    'Adam & Eve double tops': 'https://thepatternsite.com/aedt.html',
    'Adam White setup': 'https://thepatternsite.com/AdamWhiteSetup.html',
    'Area gaps': 'https://thepatternsite.com/gaps.html',
    'Ascending and inverted scallops': 'https://thepatternsite.com/aiscallop.html',
    'Ascending broadening wedge': 'https://thepatternsite.com/abw.html',
    'Ascending scallops': 'https://thepatternsite.com/ascscallop.html',
    'Ascending triangles': 'https://thepatternsite.com/at.html',
    'Ascending triangles, busted': 'https://thepatternsite.com/BustAscTriangles.html',
    'Ascending triangles, Elliott wave': 'https://thepatternsite.com/EWTriangleAscending.html',
    'Ascending triangle setup': 'https://thepatternsite.com/AscTriangleSetup.html',
    'Asset protection': 'https://thepatternsite.com/AssetProtection.html',
    'Averaging down': 'https://thepatternsite.com/AveragingDown.html',
    'Basic motive phase': 'https://thepatternsite.com/EWBasic.html',
    'Bat, bearish': 'https://thepatternsite.com/BatBear.html',
    'Bat, bullish': 'https://thepatternsite.com/BatBull.html',
    'Bearish 2-Close': 'https://thepatternsite.com/2Closebear.html',
    'Bearish AB=CD': 'https://thepatternsite.com/ABCDBear.html',
    'Bearish bat': 'https://thepatternsite.com/BatBear.html',
    'Bearish crab': 'https://thepatternsite.com/CrabBear.html',
    'Bullish crab': 'https://thepatternsite.com/CrabBull.html',
    'Bearish Gartley': 'https://thepatternsite.com/GartleyBear.html',
    'Bearish turn-key': 'https://thepatternsite.com/TurnkeyBear.html',
    'Bullish turn-key': 'https://thepatternsite.com/TurnkeyBull.html',
    'Bearish 2-step': 'https://thepatternsite.com/2StepBear.html',
    'Bullish 2-step': 'https://thepatternsite.com/2StepBull.html',
    'Bearish Wolfe wave': 'https://thepatternsite.com/WolfeWaveBear.html',
    'Bearish butterfly': 'https://thepatternsite.com/ButterflyBear.html',
    'Bearish Carl V': 'https://thepatternsite.com/CarlVBear.html',
    'Bearish double key reversal': 'https://thepatternsite.com/DoubleKeyBear.html',
    'Bearish fakey': 'https://thepatternsite.com/FakeyBear.html',
    'Best buy days': 'https://thepatternsite.com/BestBuyDays.html',
    'Best buy months': 'https://thepatternsite.com/BestBuyMonths.html',
    'Big M': 'https://thepatternsite.com/bigm.html',
    'Big W': 'https://thepatternsite.com/bigw.html',
    'Book reviews': 'https://thepatternsite.com/BookReviews.html',
    'Book value': 'https://thepatternsite.com/BookValue.html',
    'Bottoms, the nature of': 'https://thepatternsite.com/Bottoms.html',
    'Bottoms, finding market': 'https://thepatternsite.com/Bottom.html',
    'Breakaway gaps': 'https://thepatternsite.com/gaps.html',
    'Broadening bottoms': 'https://thepatternsite.com/broadb.html',
    'Broadening formations, right-angled and ascending': 'https://thepatternsite.com/rabfa.html',
    'Broadening formations, right-angled and descending': 'https://thepatternsite.com/rabfd.html',
    'Broadening tops': 'https://thepatternsite.com/bt.html',
    'Broadening wedge, ascending': 'https://thepatternsite.com/abw.html',
    'Broadening wedge, descending': 'https://thepatternsite.com/dbw.html',
    'Bullish 2-close': 'https://thepatternsite.com/2Closebull.html',
    'Bullish AB=CD': 'https://thepatternsite.com/ABCDBull.html',
    'Bullish bat': 'https://thepatternsite.com/BatBull.html',
    'Bullish butterfly': 'https://thepatternsite.com/ButterflyBull.html',
    'Bullish Carl V': 'https://thepatternsite.com/CarlVBull.html',
    'Bullish double key reversal': 'https://thepatternsite.com/DoubleKeyBull.html',
    'Bullish fakey': 'https://thepatternsite.com/FakeyBull.html',
    'Bullish Gartley': 'https://thepatternsite.com/GartleyBull.html',
    'Bullish Wolfe wave': 'https://thepatternsite.com/WolfeWaveBull.html',
    'Bump-and-run reversal bottoms': 'https://thepatternsite.com/barrb.html',
    'Bump-and-run reversal tops': 'https://thepatternsite.com/barrt.html',
    'Busted patterns': 'https://thepatternsite.com/Busted.html',
    'Busted ascending triangle': 'https://thepatternsite.com/BustAscTriangles.html',
    'Busted descending triangle': 'https://thepatternsite.com/BustDescTriangles.html',
    'Busted double bottom': 'https://thepatternsite.com/BustDoubleBots.html',
    'Busted double top': 'https://thepatternsite.com/BustDoubleTops.html',
    'Busted head-and-shoulders Bottoms': 'https://thepatternsite.com/BustHSB.html',
    'Busted head-and-shoulders Tops': 'https://thepatternsite.com/BustHST.html',
    'Busted rectangle': 'https://thepatternsite.com/BustRectangles.html',
    'Busted symmetrical triangle': 'https://thepatternsite.com/BustSymTriangles.html',
    'Busted triple bottom': 'https://thepatternsite.com/BustTripleBots.html',
    'Busted triple top': 'https://thepatternsite.com/BustTripleTops.html',
    'Butterfly, bearish': 'https://thepatternsite.com/ButterflyBear.html',
    'Butterfly, bullish': 'https://thepatternsite.com/ButterflyBull.html',
    'Carl V, bearish': 'https://thepatternsite.com/CarlVBear.html',
    'Carl V, bullish': 'https://thepatternsite.com/CarlVBull.html',
    "Cat's ears": 'https://thepatternsite.com/CatsEars.html',
    'Channels': 'https://thepatternsite.com/channels.html',
    'Channels, drawing 3-point': 'https://thepatternsite.com/3PointChannels.html',
    'Closing price reversal, downtrend': 'https://thepatternsite.com/CPRD.html',
    'Closing price reversal, uptrend': 'https://thepatternsite.com/CPRU.html',
    'Cloud banks': 'https://thepatternsite.com/Cloudbank.html',
    'Common gaps': 'https://thepatternsite.com/gaps.html',
    'Complex head-and-shoulders bottoms': 'https://thepatternsite.com/chsb.html',
    'Complex head-and-shoulders tops': 'https://thepatternsite.com/chst.html',
    'Continuation gaps': 'https://thepatternsite.com/gaps.html',
    'Corrective phase': 'https://thepatternsite.com/EWCorrective.html',
    'The CPSetup': 'https://thepatternsite.com/CPSetup.html',
    'Crab, bearish': 'https://thepatternsite.com/CrabBear.html',
    'Crab, bullish': 'https://thepatternsite.com/CrabBull.html',
    'Cup with handle': 'https://thepatternsite.com/cup.html',
    'Cup with handle, inverted': 'https://thepatternsite.com/icup.html',
    '2-Dance': 'https://thepatternsite.com/2Dance.html',
    '2-Did': 'https://thepatternsite.com/2Did.html',
    'Dead-cat bounce': 'https://thepatternsite.com/dcb.html',
    'Dead-cat bounce, inverted': 'https://thepatternsite.com/idcb.html',
    'Descending and inverted scallops': 'https://thepatternsite.com/idscallops.html',
    'Descending broadening wedges': 'https://thepatternsite.com/dbw.html',
    'Descending scallops': 'https://thepatternsite.com/descscallops.html',
    'Descending triangles': 'https://thepatternsite.com/dt.html',
    'Descending triangle, busted': 'https://thepatternsite.com/BustDescTriangles.html',
    'Descending triangle, Elliott wave': 'https://thepatternsite.com/EWTriangleDescending.html',
    'Diamond bottoms': 'https://thepatternsite.com/diamondb.html',
    'Diamond tops': 'https://thepatternsite.com/diamondt.html',
    'Divergence': 'https://thepatternsite.com/divergence.html',
    'Diving board': 'https://thepatternsite.com/DivingBoard.html',
    'Dome-shaped volume': 'https://thepatternsite.com/volshapes.html',
    'Domed house and three peaks': 'https://thepatternsite.com/3peaksdome.html',
    'Double bottoms, Adam & Adam': 'https://thepatternsite.com/aadb.html',
    'Double bottoms, Adam & Eve': 'https://thepatternsite.com/aedb.html',
    'Double bottom, busted': 'https://thepatternsite.com/BustDoubleBots.html',
    'Double bottoms, Eve & Adam': 'https://thepatternsite.com/eadb.html',
    'Double bottoms, Eve & Eve': 'https://thepatternsite.com/eedb.html',
    'Double bottoms, overview 1': 'https://thepatternsite.com/DoubleBottomTypes.html',
    'Double bottoms, overview 2': 'https://thepatternsite.com/AdamEvePatterns.html',
    'Double bottoms, ugly': 'https://thepatternsite.com/udb.html',
    'Double key reversal, bearish': 'https://thepatternsite.com/DoubleKeyBear.html',
    'Double key reversal, bullish': 'https://thepatternsite.com/DoubleKeyBull.html',
    'Double tops, Adam & Adam': 'https://thepatternsite.com/aadt.html',
    'Double tops, Adam & Eve': 'https://thepatternsite.com/aedt.html',
    'Double top, busted': 'https://thepatternsite.com/BustDoubleTops.html',
    'Double tops, Eve & Adam': 'https://thepatternsite.com/eadt.html',
    'Double tops, Eve & Eve': 'https://thepatternsite.com/eedt.html',
    'Double tops, overview': 'https://thepatternsite.com/AdamEvePatterns.html',
    'Double zigzag': 'https://thepatternsite.com/EWDoubleZigzag.html',
    'Downgrades, stock rating': 'https://thepatternsite.com/downgrade.html',
    'Downside weekly reversal': 'https://thepatternsite.com/WeeklyRevsDownside.html',
    'Down trendlines': 'https://thepatternsite.com/trenddown.html',
    'Drawing 3-point channels': 'https://thepatternsite.com/3PointChannels.html',
    'Dutch auction tender offers': 'https://thepatternsite.com/dutchep.html',
    'Earnings surprise, bad': 'https://thepatternsite.com/earnsbad.html',
    'Earnings surprise, good': 'https://thepatternsite.com/earnsgood.html',
    'Earnings flag': 'https://thepatternsite.com/earnflag.html',
    'Elevator stop': 'https://thepatternsite.com/ElevatorStop.html',
    'Elliott waves': 'https://thepatternsite.com/Elliott.html',
    'Ending diagonal triangle, (wedges)': 'https://thepatternsite.com/EWDiagTriangle.html',
    'Eve & Adam double bottoms': 'https://thepatternsite.com/eadb.html',
    'Eve & Adam double tops': 'https://thepatternsite.com/eadt.html',
    'Eve & Eve double bottoms': 'https://thepatternsite.com/eedb.html',
    'Eve & Eve double tops': 'https://thepatternsite.com/eedt.html',
    'FDA drug approvals': 'https://thepatternsite.com/fda.html',
    'Same-store sales, bad': 'https://thepatternsite.com/sssbad.html',
    'Same-store sales, good': 'https://thepatternsite.com/sssgood.html',
    'Stock rating downgrades': 'https://thepatternsite.com/downgrade.html',
    'Stock rating upgrades': 'https://thepatternsite.com/upgrade.html',
    'Ex-dividend gaps': 'https://thepatternsite.com/gaps.html',
    'Exhaustion gaps': 'https://thepatternsite.com/gaps.html',
    'Expanded flag': 'https://thepatternsite.com/EWExpanded.html',
    'Extended V-bottoms.': 'https://thepatternsite.com/vBottomExts.html',
    'Extended V-tops': 'https://thepatternsite.com/VTopExt.html',
    'Failure Swings': 'https://thepatternsite.com/failswing.html',
    'Fakey, bearish': 'https://thepatternsite.com/FakeyBear.html',
    'Fakey, bullish': 'https://thepatternsite.com/FakeyBull.html',
    'Falling volume trend': 'https://thepatternsite.com/voltrend.html',
    'Falling wedge': 'https://thepatternsite.com/fallwedge.html',
    'FDA drug approvals': 'https://thepatternsite.com/fda.html',
    'Finding market bottoms': 'https://thepatternsite.com/Bottom.html',
    'First wave extension': 'https://thepatternsite.com/EWExtension1.html',
    'Fifth wave extension': 'https://thepatternsite.com/EWExtension5.html',
    'Flags': 'https://thepatternsite.com/flags.html',
    'Flags, earnings': 'https://thepatternsite.com/earnflag.html',
    'Flags, high and tight': 'https://thepatternsite.com/htf.html',
    'Flat (3-3-5)': 'https://thepatternsite.com/EWFlat.html',
    'Flat base': 'https://thepatternsite.com/FlatBase.html',
    'Flat, expanded': 'https://thepatternsite.com/EWExpanded.html',
    'Flat, running': 'https://thepatternsite.com/EWRunning.html',
    'Gap2H': 'https://thepatternsite.com/Gap2H.html',
    'Gap2H, inverted': 'https://thepatternsite.com/Gap2Hi.html',
    'Gaps': 'https://thepatternsite.com/gaps.html',
    'Gaps, area': 'https://thepatternsite.com/gaps.html',
    'Gaps, breakaway': 'https://thepatternsite.com/gaps.html',
    'Gaps, common': 'https://thepatternsite.com/gaps.html',
    'Gaps, continuation': 'https://thepatternsite.com/gaps.html',
    'Gaps, ex-dividend': 'https://thepatternsite.com/gaps.html',
    'Gaps, exhaustion': 'https://thepatternsite.com/gaps.html',
    'Gaps, measuring': 'https://thepatternsite.com/gaps.html',
    'Gaps, runaway': 'https://thepatternsite.com/gaps.html',
    'Gartley, bearish': 'https://thepatternsite.com/GartleyBear.html',
    'Gartley, bullish': 'https://thepatternsite.com/GartleyBull.html',
    'Head-and-shoulders bottoms': 'https://thepatternsite.com/hsb.html',
    'Head-and-shoulders bottoms, busted': 'https://thepatternsite.com/BustHSB.html',
    'Head-and-shoulders complex bottoms': 'https://thepatternsite.com/chsb.html',
    'Head-and-shoulders complex tops': 'https://thepatternsite.com/chst.html',
    'Head-and-shoulders tops': 'https://thepatternsite.com/hst.html',
    'Head-and-shoulders tops, busted': 'https://thepatternsite.com/BustHST.html',
    'High and tight flags': 'https://thepatternsite.com/htf.html',
    'Hook reversal, downtrend': 'https://thepatternsite.com/HRD.html',
    'Hook reversal, uptrend': 'https://thepatternsite.com/HRU.html',
    'Horn bottoms': 'https://thepatternsite.com/hornb.html',
    'Horn tops': 'https://thepatternsite.com/hornt.html',
    'Inside days': 'https://thepatternsite.com/InsideDays.html',
    'Inverted 3L-R': 'https://thepatternsite.com/3L-Ri.html',
    'Inverted and ascending scallops': 'https://thepatternsite.com/aiscallop.html',
    'Inverted and descending scallops': 'https://thepatternsite.com/idscallops.html',
    'Inverted cup with handle': 'https://thepatternsite.com/icup.html',
    'Inverted dead-cat bounce': 'https://thepatternsite.com/idcb.html',
    'Inverted gap2H': 'https://thepatternsite.com/Gap2Hi.html',
    'Inverted roof': 'https://thepatternsite.com/iroof.html',
    'Inverted V pivot': 'https://thepatternsite.com/InvVPivot.html',
    'Island bottoms': 'https://thepatternsite.com/islandrev.html',
    'Island, long': 'https://thepatternsite.com/longisland.html',
    'Island reversal': 'https://thepatternsite.com/islandrev.html',
    'Island tops': 'https://thepatternsite.com/islandrev.html',
    'Key reversal, downtrend': 'https://thepatternsite.com/KRD.html',
    'Key reversal bar, bullish, v2': 'https://thepatternsite.com/KRB2.html',
    'Key reversal bar, bearish, v2': 'https://thepatternsite.com/KRB2Bear.html',
    'Key reversal, uptrend': 'https://thepatternsite.com/KRU.html',
    'Knots': 'https://thepatternsite.com/TradingKnots.html',
    'Leading diagonal triangle (wedges)': 'https://thepatternsite.com/EWleadingTriangle.html',
    'Long island': 'https://thepatternsite.com/longisland.html',
    'Measured move down': 'https://thepatternsite.com/mmd.html',
    'Measured move up': 'https://thepatternsite.com/mmu.html',
    'Measuring gaps': 'https://thepatternsite.com/gaps.html',
    'Minor highs and lows': 'https://thepatternsite.com/minorhl.html',
    'Mirrors, price': 'https://thepatternsite.com/Mirrors.html',
    'Monthly symmetrical triangles': 'https://thepatternsite.com/msymtri.html',
    'Monthly trends (channels).': 'https://thepatternsite.com/MonthlyChannels.html',
    'Motive wave': 'https://thepatternsite.com/EWBasic.html',
    'Mountain, price': 'https://thepatternsite.com/mountain.html',
    'Multi-peaks.': 'https://thepatternsite.com/MultiPeaks.html',
    'Nr4 (Narrow range 4)': 'https://thepatternsite.com/NR4.html',
    'Nr7 (Narrow range 7)': 'https://thepatternsite.com/nr7.html',
    'The nature of bottoms': 'https://thepatternsite.com/Bottoms.html',
    'One day reversal, bottom': 'https://thepatternsite.com/ODRB.html',
    'One day reversal, top': 'https://thepatternsite.com/ODRT.html',
    'Open-close reversal, downtrend': 'https://thepatternsite.com/OCRD.html',
    'Open-close reversal, uptrend': 'https://thepatternsite.com/OCRU.html',
    'Outside Days': 'https://thepatternsite.com/OutsideDays.html',
    'Partial Declines': 'https://thepatternsite.com/partdecline.html',
    'Partial Rises': 'https://thepatternsite.com/partrises.html',
    'Pennants': 'https://thepatternsite.com/pennants.html',
    'Performance, rank': 'https://thepatternsite.com/rank.html',
    'Performance, time': 'https://thepatternsite.com/TimePerformance.html',
    'Pipe bottoms': 'https://thepatternsite.com/pipeb.html',
    'Pipe tops': 'https://thepatternsite.com/pipet.html',
    'Pivot point reversal, downtrend': 'https://thepatternsite.com/PPRD.html',
    'Pivot point reversal, uptrend': 'https://thepatternsite.com/PPRU.html',
    'Pivot, V': 'https://thepatternsite.com/VPivot.html',
    'Pothole': 'https://thepatternsite.com/Pothole.html',
    'Price mirrors': 'https://thepatternsite.com/Mirrors.html',
    'Price mountain': 'https://thepatternsite.com/mountain.html',
    'Pullbacks': 'https://thepatternsite.com/pullbacks.html',
    'Random-shaped volume': 'https://thepatternsite.com/volshapes.html',
    'Rank': 'https://thepatternsite.com/rank.html',
    'Rank over time': 'https://thepatternsite.com/TimePerformance.html',
    'Rectangle bottoms': 'https://thepatternsite.com/rectbots.html',
    'Rectangle, busted': 'https://thepatternsite.com/BustRectangles.html',
    'Rectangle tops': 'https://thepatternsite.com/recttops.html',
    'Resistance': 'https://thepatternsite.com/SAR.html',
    'Reverse symmetrical Triangle': 'https://thepatternsite.com/EWRevSymmetrical.html',
    'Right-angled and ascending broadening formations': 'https://thepatternsite.com/rabfa.html',
    'Right-angled and descending broadening formations': 'https://thepatternsite.com/rabfd.html',
    'Rising volume trend': 'https://thepatternsite.com/voltrend.html',
    'Rising wedge': 'https://thepatternsite.com/risewedge.html',
    'Roof': 'https://thepatternsite.com/roof.html',
    'Roof, inverted': 'https://thepatternsite.com/iroof.html',
    'Rounding bottoms': 'https://thepatternsite.com/roundb.html',
    'Rounding tops': 'https://thepatternsite.com/roundingtop.html',
    'Runaway gaps': 'https://thepatternsite.com/gaps.html',
    'Running flat': 'https://thepatternsite.com/EWRunning.html',
    'Running triangle': 'https://thepatternsite.com/EWTriangleRunning.html',
    'Same-store sales, bad': 'https://thepatternsite.com/sssbad.html',
    'Same-store sales, good': 'https://thepatternsite.com/sssgood.html',
    'Scallops, ascending': 'https://thepatternsite.com/ascscallop.html',
    'Scallops, ascending and inverted': 'https://thepatternsite.com/aiscallop.html',
    'Scallops, descending': 'https://thepatternsite.com/descscallops.html',
    'Scallops, descending and inverted': 'https://thepatternsite.com/idscallops.html',
    'Shark-32': 'https://thepatternsite.com/Shark32.html',
    'Simple ABC correction': 'https://thepatternsite.com/abc.html',
    'Slider quizzes': 'https://thepatternsite.com/quiz.html',
    'Small patterns': 'https://thepatternsite.com/SmallPatterns.html',
    '2-close bullish reversal': 'https://thepatternsite.com/2Closebull.html',
    '2-Close bearish reversal': 'https://thepatternsite.com/2Closebear.html',
    '2-dance': 'https://thepatternsite.com/2Dance.html',
    '2-did': 'https://thepatternsite.com/2Did.html',
    '2-step, bearish': 'https://thepatternsite.com/2StepBear.html',
    '2-step, bullish': 'https://thepatternsite.com/2StepBull.html',
    '2-tall': 'https://thepatternsite.com/TallDance.html',
    '3-Bar': 'https://thepatternsite.com/3Bar.html',
    '3DC': 'https://thepatternsite.com/3DC.html',
    '3L-R': 'https://thepatternsite.com/3L-R.html',
    '3L-R, inverted': 'https://thepatternsite.com/3L-Ri.html',
    'Closing price reversal, downtrend': 'https://thepatternsite.com/CPRD.html',
    'Closing price reversal, uptrend': 'https://thepatternsite.com/CPRU.html',
    'Double key reversal, bearish': 'https://thepatternsite.com/DoubleKeyBear.html',
    'Double key reversal, bullish': 'https://thepatternsite.com/DoubleKeyBull.html',
    'Fakey, bearish': 'https://thepatternsite.com/FakeyBear.html',
    'Fakey, bullish': 'https://thepatternsite.com/FakeyBull.html',
    'Gap2H': 'https://thepatternsite.com/Gap2H.html',
    'Gap2H, inverted': 'https://thepatternsite.com/Gap2Hi.html',
    'Hook reversal, downtrend': 'https://thepatternsite.com/HRD.html',
    'Hook reversal, uptrend': 'https://thepatternsite.com/HRU.html',
    'Inside Days': 'https://thepatternsite.com/InsideDays.html',
    'Key reversal, downtrend': 'https://thepatternsite.com/KRD.html',
    'Key reversal, uptrend': 'https://thepatternsite.com/KRU.html',
    'Key reversal bar, bullish, v2': 'https://thepatternsite.com/KRB2.html',
    'Key reversal bar, bearish, v2': 'https://thepatternsite.com/KRB2Bear.html',
    'Narrow range 4': 'https://thepatternsite.com/NR4.html',
    'Narrow range 7': 'https://thepatternsite.com/nr7.html',
    'One day reversal, bottom': 'https://thepatternsite.com/ODRB.html',
    'One day reversal, top': 'https://thepatternsite.com/ODRT.html',
    'Open-close reversal, downtrend': 'https://thepatternsite.com/OCRD.html',
    'Open-close reversal, uptrend': 'https://thepatternsite.com/OCRU.html',
    'Outside Days': 'https://thepatternsite.com/OutsideDays.html',
    'Pivot point reversal, downtrend': 'https://thepatternsite.com/PPRD.html',
    'Pivot point reversal, uptrend': 'https://thepatternsite.com/PPRU.html',
    'Shark-32': 'https://thepatternsite.com/Shark32.html',
    'Turn-key, bearish': 'https://thepatternsite.com/TurnkeyBear.html',
    'Turn-key, bullish': 'https://thepatternsite.com/TurnkeyBull.html',
    'Weekly reversals, downside': 'https://thepatternsite.com/WeeklyRevsDownside.html',
    'Weekly reversals, upside': 'https://thepatternsite.com/WeeklyRevsUpside.html',
    'Wide ranging day downside reversal': 'https://thepatternsite.com/WRDDR.html',
    'Wide ranging day upside reversal': 'https://thepatternsite.com/WRDUR.html',
    'Stop, elevator': 'https://thepatternsite.com/ElevatorStop.html',
    'Stop, volatility': 'https://thepatternsite.com/stops.html',
    'Spikes': 'https://thepatternsite.com/spikes.html',
    'Support': 'https://thepatternsite.com/SAR.html',
    'Stock downgrades': 'https://thepatternsite.com/downgrade.html',
    'Stock upgrades': 'https://thepatternsite.com/upgrade.html',
    'Symmetrical triangles': 'https://thepatternsite.com/st.html',
    'Symmetrical triangle, Elliott wave': 'https://thepatternsite.com/EWTriangleSymmetrical.html',
    'Symmetrical triangles, monthly': 'https://thepatternsite.com/msymtri.html',
    'Tails': 'https://thepatternsite.com/spikes.html',
    'Ten baggers': 'https://thepatternsite.com/10Baggers.html',
    '(Three-bar) 3-Bar': 'https://thepatternsite.com/3Bar.html',
    'Three falling peaks': 'https://thepatternsite.com/3fp.html',
    'Three peaks and domed house': 'https://thepatternsite.com/3peaksdome.html',
    'Three peaks and spike': 'https://thepatternsite.com/MultiPeak2B.html',
    'Three rising valleys': 'https://thepatternsite.com/3rv.html',
    'Third wave extension': 'https://thepatternsite.com/EWExtension3.html',
    'Throwbacks': 'https://thepatternsite.com/throwbacks.html',
    'Time performance': 'https://thepatternsite.com/TimePerformance.html',
    'Trend change, 1-2-3': 'https://thepatternsite.com/123tc.html',
    'Trend compression, 3-day (3DC)': 'https://thepatternsite.com/3DC.html',
    'Trendlines, down': 'https://thepatternsite.com/trenddown.html',
    'Trendlines, up': 'https://thepatternsite.com/uptrendlines.html',
    'Trends (channels), monthly.': 'https://thepatternsite.com/MonthlyChannels.html',
    'Triangles, ascending': 'https://thepatternsite.com/at.html',
    'Triangle, ascending, busted': 'https://thepatternsite.com/BustAscTriangles.html',
    'Triangle, ascending, Elliott wave': 'https://thepatternsite.com/EWTriangleAscending.html',
    'Triangles, descending': 'https://thepatternsite.com/dt.html',
    'Triangle, descending, busted': 'https://thepatternsite.com/BustDescTriangles.html',
    'Triangle, descending, Elliott wave': 'https://thepatternsite.com/EWTriangleDescending.html',
    'Triangle, ending diagonal (wedges)': 'https://thepatternsite.com/EWDiagTriangle.html',
    'Triangle, leading diagonal (wedges)': 'https://thepatternsite.com/EWleadingTriangle.html',
    'Triangle, reverse symmetrical (broadening)': 'https://thepatternsite.com/EWRevSymmetrical.html',
    'Triangle, running': 'https://thepatternsite.com/EWTriangleRunning.html',
    'Triangles, symmetrical': 'https://thepatternsite.com/st.html',
    'Triangle, symmetrical, busted': 'https://thepatternsite.com/BustSymTriangles.html',
    'Triangles, symmetrical, monthly': 'https://thepatternsite.com/msymtri.html',
    'Triangle, symmetrical, Elliott wave': 'https://thepatternsite.com/EWTriangleSymmetrical.html',
    'Triple bottoms': 'https://thepatternsite.com/tb.html',
    'Triple bottom, busted': 'https://thepatternsite.com/BustTripleBots.html',
    'Triple tops': 'https://thepatternsite.com/tt.html',
    'Triple top, busted': 'https://thepatternsite.com/BustTripleTops.html',
    'Truncation': 'https://thepatternsite.com/EWTruncation.html',
    'Turn-key, bearish': 'https://thepatternsite.com/TurnkeyBear.html',
    'Turn-key, bullish': 'https://thepatternsite.com/TurnkeyBull.html',
    '(Two-dance) 2-dance': 'https://thepatternsite.com/2Dance.html',
    '(Two did) 2-did': 'https://thepatternsite.com/2Did.html',
    '(Two-step) 2-step, bearish': 'https://thepatternsite.com/2StepBear.html',
    '(Two-step) 2-step, bullish': 'https://thepatternsite.com/2StepBull.html',
    '(Two-tall) 2-tall': 'https://thepatternsite.com/TallDance.html',
    'Ugly double bottoms': 'https://thepatternsite.com/udb.html',
    'Unknown wave extension': 'https://thepatternsite.com/EWExtensionU.html',
    'Upgrades, stock rating': 'https://thepatternsite.com/upgrade.html',
    'Upside weekly reversal': 'https://thepatternsite.com/WeeklyRevsUpside.html',
    'Up trendlines': 'https://thepatternsite.com/uptrendlines.html',
    'U-shaped volume': 'https://thepatternsite.com/volshapes.html',
    'V-bottoms': 'https://thepatternsite.com/vBottoms.html',
    'V-bottoms, extended': 'https://thepatternsite.com/vBottomExts.html',
    'V tops': 'https://thepatternsite.com/VTop.html',
    'V-tops, extended': 'https://thepatternsite.com/VTopExt.html',
    'Vertical run down': 'https://thepatternsite.com/VerticalRunDown.html',
    'Vertical run up': 'https://thepatternsite.com/VerticalRunUp.html',
    'V pivot': 'https://thepatternsite.com/VPivot.html',
    'V pivot, inverted': 'https://thepatternsite.com/InvVPivot.html',
    'Volatility stop': 'https://thepatternsite.com/stops.html',
    'Volume, breakout day': 'https://thepatternsite.com/volbkout.html',
    'Volume, dome-shaped': 'https://thepatternsite.com/volshapes.html',
    'Volume, random shaped': 'https://thepatternsite.com/volshapes.html',
    'Volume shapes': 'https://thepatternsite.com/volshapes.html',
    'Volume trend': 'https://thepatternsite.com/voltrend.html',
    'Volume, U-shaped': 'https://thepatternsite.com/volshapes.html',
    'Wave extension, first': 'https://thepatternsite.com/EWExtension1.html',
    'Wave extension, third': 'https://thepatternsite.com/EWExtension3.html',
    'Wave extension, fifth': 'https://thepatternsite.com/EWExtension5.html',
    'Wave extension, unknown': 'https://thepatternsite.com/EWExtensionU.html',
    'Wedge, ascending broadening': 'https://thepatternsite.com/abw.html',
    'Wedge, descending broadening': 'https://thepatternsite.com/dbw.html',
    'Wedge, falling': 'https://thepatternsite.com/fallwedge.html',
    'Wedge, rising': 'https://thepatternsite.com/risewedge.html',
    'Weekly reversal, downside': 'https://thepatternsite.com/WeeklyRevsDownside.html',
    'Weekly reversal, upside': 'https://thepatternsite.com/WeeklyRevsUpside.html',
    'Wide ranging day downside reversal': 'https://thepatternsite.com/WRDDR.html',
    'Wide ranging day upside reversal': 'https://thepatternsite.com/WRDUR.html',
    'Wolfe wave, bearish': 'https://thepatternsite.com/WolfeWaveBear.html',
    'Wolfe wave, bullish': 'https://thepatternsite.com/WolfeWaveBull.html',
    'Zigzag': 'https://thepatternsite.com/EWZigzag.html',
    'Zigzag, double': 'https://thepatternsite.com/EWDoubleZigzag.html'
}

In [36]:
notes = {
    "1-2-3 trend change": "Use a zig-zag type indicator with a threshold for detecting trend changes. You must also have a window size that specifies the number of bars in each analysis, and you must use the sliding window. The results will be a Series of booleans that will indicate whether the pattern was first evident at that bar.",
    "2B": "Use a zig-zag type indicator with a threshold for detecting trend changes.",
    "3 falling peaks": "Use a zig-zag type indicator with a threshold for detecting trend changes.",
    "3L-R": "Base your calculations on a zig-zag type indicator with a threshold for detecting direction changes.",
    '2-did': "Rather than basing the length of the bars on the prior month, they should be gauged against a base period (with a parameter) that defaults to a month long (22 trading days)",
}

In [35]:
skip = 5
do_n = 2
print(f"len(patterns) = {len(patterns)}")

len(patterns) = 384


In [39]:
module_stub_code = generate_module_code_stubs(ta_parser)


n_done = 0
skipped = 0
n_on = 0
for line in patterns:
    n_on = n_on + 1    
    if(len(line) > 1):
        if(skipped < skip):
            skipped = skipped + 1
            continue
        if line[0] == '!':
            continue
        url = patterns[line]
        note = notes.get(line, "none")
        page = get_url_content(url)[:512*1024*1024]
        question = f"""
I am working with a Jupyter notebook that has defined the following classes and functions\n
(note: I am showing stubs here, but the code is largely implemented):

--[
{module_stub_code}
]--

This is HTML documenting how a pattern detection algorithm should work:

--[
{page}
]--
Please observe these notes: 
{note}

Write a function that will detect the '{line}' pattern.
The function must match the form of the do_calculate_xxx() functions and be named do_detect_xxx().
It should receive a dataframe with "open" "high" "low" "close" "volume" and "date".
Do NOT perform unnecessary validity checks on the dataframe.
Do NOT return an empty Series unless the dataframe has 0 rows.
The index MUST be retained in the return value.
Do NOT modify the dataframe must not be modified.
I repeat: Do NOT return an empty Series(). The Series MUST have the same index as the dataframe so it can be used with .join().
Your commentary MUST be Python comments placed BEFORE the function.
Do not include any imports in your response.
Do not perform unnecessary validity checks or transformations or comment about htem.
Do NOT do fillna(0).
Place three newlines after each function.
Do not acknowledge these instructions in your comments; it's a waste of space.
Do not make unnecessary and comments about the function of trivial Python expressions such as .apply().
Do not comment on the index alignment; it is ALWAYS assumed that the index should be aligned. 
Do not comment on ordinary measures to deal with edge cases
If you believe that the generated code will not work or that it needs significant modification, please include a FIXME: line in the docstring. Under no circumstances may you modify the dataframe parameter; it should be marked as constant if possible.
Please include a docstring.
All parameters must have reasonable defaults except for the dataframe.
Please do not comment on the predictive power or rarity of the pattern. This information is possibly unreliable.
This line must be included as the first line of your comments: Ref: {url}
It is not permitted to make any assignments to the df parameter or any part of the underlying object. For example, commands like
df['is_white'] = df['close'] > df['open'] are illegal. You must use your own Series variables in those sorts of situations. The parameter to shift() should
never be negative as that would imply using data from the future.
NEVER USE ANY NEGATIVE PARAMETER WITH THE .shift() METHOD or you will kill penguins as more GPU cycles are wasted to fix your error.
Avoid hardcoded constants; any thresholds should be specified as parameters.
Please use fractions instead of percentages.
If for some reason you are unable to generate a pattern matching function (for example, if the page does not describe a pattern that uses price data and
is straightforwardly detected as a Series of True / False values) please stub
out the function with a FIXME line and explain the issue. If it is necessary to add a window_size parameter and use rolling(), please do. Please do not combine
bullish and bearish pattern detectors in the same function. If it is necessary, create both a bull and bear version.
Even if you have not placed any FIXME lines, add a FIXME line to the comments saying that this function needs to be manually checked.
"""        
        if(len(notes) > 0):
            question = question + f"\nNotes:\n{notes}"
        response = gemini_model.generate_content(question)
        print(f"\n# [{n_on} of {len(patterns)}]")
        print(f"#'{line}'")
        display(ipyd.Markdown(response.text))
        print("\n\n\n")
        time.sleep(1)
        n_done = n_done + 1            
        if n_done >= do_n:
            break

skip = skip + do_n


# [10 of 384]
#'3 falling peaks'


```python
# Ref: https://thepatternsite.com/3fp.html
# FIXME: This function needs to be manually checked.

def do_detect_three_falling_peaks(df: pd.DataFrame, peak_threshold: float = 0.02, valley_threshold: float = 0.01, window_size: int = 3):
    """
    Detects the Three Falling Peaks candlestick pattern.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').
        peak_threshold: Minimum percentage change between consecutive highs to be considered a peak.
        valley_threshold: Minimum percentage change between consecutive lows to be considered a valley.
        window_size: The number of bars to consider for the pattern detection.

    Returns:
        pd.Series: Boolean Series indicating Three Falling Peaks patterns.
        FIXME: Requires testing and refinement of threshold parameters.
    """
    if len(df) == 0:
        return pd.Series([], dtype=bool)

    highs = df['high']
    lows = df['low']

    # Calculate percentage changes for highs and lows
    high_changes = highs.pct_change()
    low_changes = lows.pct_change()

    # Identify peaks and valleys
    peaks = (high_changes > peak_threshold).astype(int)
    valleys = (low_changes < -valley_threshold).astype(int)

    # Apply sliding window to detect pattern
    is_three_falling_peaks = (peaks.rolling(window=window_size, center=False).sum() == 3) & (lows.shift(window_size-1) > lows)

    return is_three_falling_peaks


```



```python

```



```python

```







# [11 of 384]
#'3L-R'


```python
# Ref: https://thepatternsite.com/3L-R.html
# FIXME: This function needs to be manually checked.

def do_detect_3l_r(df: 'pd.DataFrame', lower_low_threshold: float = 0.01, higher_high_threshold: float = 0.01):
    """
    Detects the 3L-R candlestick pattern.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').
        lower_low_threshold: Minimum percentage decrease in consecutive lows to be considered a lower low.
        higher_high_threshold: Minimum percentage increase in the high of the last bar compared to the first bar.

    Returns:
        pd.Series: Boolean Series indicating 3L-R patterns.
    """
    lows = df['low']
    highs = df['high']

    # Detect consecutive lower lows
    lower_lows = lows.pct_change() < -lower_low_threshold
    
    #Find groups of at least two consecutive lower lows
    groups = lower_lows.groupby((~lower_lows).cumsum()).cumcount() + 1
    
    #Check for groups of at least two lower lows
    three_lower_lows = groups >= 2
    
    #Check if the last bar has a high above the first bar of the group
    higher_high = (highs.shift(-3) - highs) / highs * 100 > higher_high_threshold

    #Combine conditions to detect 3L-R
    pattern = three_lower_lows & higher_high
    
    #Ensure the result is aligned with the original dataframe index
    return pattern


```



```python

```

```python

```

In [25]:
# by ChatGPT with much prompting and minimal editing
def calculate_kelly_bet_size(returns, weights=None):
    """
    Calculate the optimal bet size using the generalized Kelly criterion, 
    given a list of ordinary returns and weights.

    Parameters:
    - returns: List of ordinary returns for each outcome. No subtraction of 1 (just raw price ratio).
    - weights: List of weights (probabilities) corresponding to each return. 
              If None, assumes equal weights.

    Returns:
    - optimal_bet_size: The optimal fraction of bankroll to bet.
    """
    # Convert inputs to numpy arrays for easier manipulation
    returns = np.array(returns)

    # If weights are not provided, assume equal weights
    if weights is None:
        weights = np.ones(len(returns)) / len(returns)
    else:
        weights = np.array(weights)

    # Function to compute the expected logarithmic growth using ordinary returns
    def expected_log_growth(f):
        # Calculate the wealth after each bet using simple returns (R_i)
        wealth_after_bet = weights * np.log(1 + f * (returns - 1))  # Correct formula
        return np.sum(wealth_after_bet)

    # Use optimization to find the optimal bet size
    result = scipy.optimize.minimize_scalar(lambda f: -expected_log_growth(f), bounds=(0, 1), method='bounded')

    optimal_bet_size = result.x  # This is the optimal bet size that maximizes expected log growth
    return optimal_bet_size


In [26]:
universe = df.loc[df["ticker"] == "SPY"]
display(universe)

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated,"MACD(12,26,9)[""macd""]","MACD(12,26,9)[""signal""]","MACD(12,26,9)[""histogram""]",(RSI(14)>=80)
11568775,SPY,1997-12-31,96.875,97.625,96.688,97.062,4359500.0,60.193,97.062,2025-03-24,6.960871,1.467412,5.493459,True
11568776,SPY,1998-01-02,97.312,97.656,96.531,97.562,2360900.0,60.503,97.562,2025-03-24,12.298026,3.633535,8.664491,True
11568777,SPY,1998-01-05,97.844,98.438,96.781,97.781,4191800.0,60.638,97.781,2025-03-24,16.356879,6.178203,10.178676,True
11568778,SPY,1998-01-06,97.250,97.281,96.188,96.219,3154900.0,59.669,96.219,2025-03-24,19.225885,8.787740,10.438145,True
11568779,SPY,1998-01-07,96.094,96.719,95.219,96.469,4424200.0,59.825,96.469,2025-03-24,21.274527,11.285097,9.989430,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11575644,SPY,2025-04-23,540.430,545.430,533.880,535.420,90590658.0,535.420,535.420,2025-04-24,-10.663216,-12.070607,1.407391,False
11575645,SPY,2025-04-24,536.715,547.430,535.450,546.690,64150418.0,546.690,546.690,2025-04-25,-8.624161,-11.381318,2.757157,False
11575646,SPY,2025-04-25,546.650,551.050,543.690,550.640,61119590.0,550.640,550.640,2025-04-28,-6.613229,-10.427700,3.814471,False
11575647,SPY,2025-04-28,551.390,553.550,545.020,550.850,47613755.0,550.850,550.850,2025-04-29,-4.945597,-9.331279,4.385682,False


In [27]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [28]:

v = factory.parse("(RSI(14) > 30) && (Shift(RSI(14), 1) < 30)").calculate(universe)

display(v.tail(100))


11575549    False
11575550    False
11575551    False
11575552    False
11575553    False
            ...  
11575644    False
11575645    False
11575646    False
11575647    False
11575648    False
Name: ((RSI(14)>30)&&(Shift(RSI(14),1)<30)), Length: 100, dtype: bool